In [ ]:
# Estimate atoms slowed to a target window using a Maxwell-Boltzmann beam model
advanced_state = globals().get("_advanced_state")
if not advanced_state:
    raise RuntimeError("Run cell 9 (advanced Zeeman slower widget) before executing this cell so the magnetic profile is available.")

shared_intensity_slider = globals().get("intensity_slider_final")
shared_detuning_slider = globals().get("detuning_slider_final")
ideal_detuning = float(advanced_state.get("delta_ideal", 0.0))
detuning_center_ghz = ideal_detuning / (2 * math.pi * 1e9) if ideal_detuning else 0.0
detuning_span_ghz = max(0.5, abs(detuning_center_ghz) * 0.5) if detuning_center_ghz else 0.5
detuning_min = detuning_center_ghz - detuning_span_ghz / 2.0
detuning_max = detuning_center_ghz + detuning_span_ghz / 2.0
fallback_controls = []
if shared_intensity_slider is None or shared_detuning_slider is None:
    fallback_controls.append(
        w.HTML(
            "<b>Local intensity/detuning controls</b>: run the final-velocity diagnostics cell to sync with its sliders.",
        )
    )
if shared_intensity_slider is None:
    shared_intensity_slider = w.FloatSlider(
        description="I (mW/cm²)",
        value=600.0,
        min=10.0,
        max=2000.0,
        step=10.0,
        continuous_update=False,
    )
    fallback_controls.append(shared_intensity_slider)
if shared_detuning_slider is None:
    shared_detuning_slider = w.FloatSlider(
        description="Detu/2&pi; (GHz)",
        value=detuning_center_ghz,
        min=detuning_min,
        max=detuning_max,
        step=0.005,
        continuous_update=False,
    )
    fallback_controls.append(shared_detuning_slider)

intensity_slider_final = shared_intensity_slider
detuning_slider_final = shared_detuning_slider

species_name = advanced_state.get("species", species.value)
species_props = advanced_state.get("species_props", species_data.get(species_name, {}))
fallback_props = species_data.get(species_name, species_data[species.value])
spec = {**fallback_props, **species_props}

wavelength = spec["wavelength"]
gamma = spec["gamma"]
mass = spec["mass"] * amu
sat_in = spec.get("sat_intensity", 60.0)
k = 2 * math.pi / wavelength
k_B = 1.380649e-23

params = advanced_state.get("params", {})
g_eff_val = float(params.get("g_eff", g_eff_adv.value if "g_eff_adv" in globals() else 1.0))
v_capture = float(params.get("v0", 500.0))
v_final = float(params.get("vf", 50.0))

z_axis_core = np.asarray(advanced_state["z"])
B_profile_core = np.asarray(advanced_state["B"])
z_profile = np.asarray(advanced_state.get("z_extended", z_axis_core))
B_profile_extended = np.asarray(advanced_state.get("B_extended", B_profile_core))
L_design = float(advanced_state.get("L", z_axis_core[-1] - z_axis_core[0]))
if z_profile.size == 0 or B_profile_extended.size == 0:
    raise RuntimeError("Advanced magnetic profile is empty. Re-run cell 9 with valid parameters.")

z_min, z_max = float(z_profile[0]), float(z_profile[-1])
flight_time = L_design / max(v_capture, 1.0)
flight_time = max(flight_time, 1e-6)
base_dt = flight_time / max(len(z_axis_core) - 1, 1)
base_steps = max(len(z_profile), 2)
min_speed = 1e-6

initial_velocity_grid = np.linspace(max(1.0, v_final * 0.2), v_capture, 250)
window_slider_max = max(150.0, v_capture * 1.2)

temperature_slider_mb = w.FloatSlider(
    description="Beam T (K)",
    value=700.0,
    min=100.0,
    max=1500.0,
    step=10.0,
    continuous_update=False,
 )
flux_slider_mb = w.FloatLogSlider(
    description="Atom flux (1/s)",
    value=1e9,
    base=10,
    min=4,
    max=12,
    step=0.1,
    continuous_update=False,
 )
window_slider_mb = w.FloatRangeSlider(
    description="Final v window (m/s)",
    value=(30.0, 60.0),
    min=0.0,
    max=window_slider_max,
    step=1.0,
    continuous_update=False,
 )
mb_summary_label = w.HTML()

def maxwell_speed_pdf(speed, particle_mass, temperature):
    beta = particle_mass / (2.0 * k_B * max(temperature, 1e-6))
    prefactor = 4.0 * math.pi * (beta / math.pi) ** 1.5
    return prefactor * (speed ** 2) * np.exp(-beta * speed ** 2)

def estimate_atoms(temperature_K, atom_flux_per_s, velocity_window):
    intensity_val = float(intensity_slider_final.value)
    detuning_val = float(detuning_slider_final.value)
    window_low, window_high = velocity_window
    if window_high <= window_low:
        mb_summary_label.value = "<span style='color:red'>Velocity window must have max > min.</span>"
        return

    s0_val = intensity_val / sat_in if sat_in else 0.0
    delta_laser = detuning_val * 1e9

    def scattering_force_vector(B_T, velocities):
        delta = delta_laser + k * velocities + g_eff_val * mu * B_T / hbar
        denom = 1.0 + s0_val + (2.0 * delta / gamma) ** 2
        return hbar * k * gamma / 2.0 * (s0_val / denom)

    def integrate_to_exit(v_init):
        speed_scale = max(1.0, v_capture / max(abs(v_init), min_speed))
        max_steps = int(math.ceil(base_steps * speed_scale * 8.0))
        v_curr = v_init
        z_curr = z_min
        final_v = v_curr
        for _ in range(max_steps):
            if abs(v_curr) < min_speed:
                v_curr = np.copysign(min_speed, final_v if abs(final_v) > 1e-9 else 1.0)
            z_sample = float(np.clip(z_curr, z_min, z_max))
            B_val = np.interp(z_sample, z_profile, B_profile_extended)
            force = scattering_force_vector(B_val, v_curr)
            accel = -force / mass
            v_next = v_curr + accel * base_dt
            z_next = z_curr + v_curr * base_dt
            final_v = v_next
            v_curr, z_curr = v_next, z_next
            if z_next >= z_max:
                break
        return final_v

    final_velocities = np.array([integrate_to_exit(v0) for v0 in initial_velocity_grid])
    pdf_vals = maxwell_speed_pdf(initial_velocity_grid, mass, temperature_K)
    dv = np.gradient(initial_velocity_grid)
    prob_weights = pdf_vals * dv
    captured_prob = prob_weights.sum()

    window_mask = (final_velocities >= window_low) & (final_velocities <= window_high)
    prob_window = prob_weights[window_mask].sum()
    atoms_in_window = atom_flux_per_s * prob_window
    captured_flux = atom_flux_per_s * captured_prob
    efficiency = (prob_window / captured_prob * 100.0) if captured_prob > 0 else 0.0

    upper_plot = max(window_high * 1.5, float(np.nanmax(final_velocities)) * 1.05, 100.0)
    bins = np.linspace(0.0, upper_plot, 40)
    hist, edges = np.histogram(final_velocities, bins=bins, weights=prob_weights)
    centers = 0.5 * (edges[:-1] + edges[1:])

    fig, ax = plt.subplots(figsize=(7, 3.5))
    ax.bar(centers, hist, width=np.diff(edges), color="tab:blue", alpha=0.7, label="Final v probability mass")
    ax.axvspan(window_low, window_high, color="tab:orange", alpha=0.2, label="Target window")
    ax.set_xlabel("Final velocity (m/s)")
    ax.set_ylabel("Probability mass")
    ax.set_title("Maxwell-Boltzmann weighted final velocity distribution")
    ax.grid(True, ls=":", alpha=0.4)
    ax.legend(loc="upper right", fontsize=8)
    plt.tight_layout()
    plt.show()

    mb_summary_label.value = (
        f"<b>Beam temperature</b>: {temperature_K:.0f} K"
        f"<br><b>Total atom flux</b>: {atom_flux_per_s:.2e} s⁻¹"
        f"<br><b>Captured flux (v ≤ {v_capture:.0f} m/s)</b>: {captured_flux:.2e} s⁻¹"
        f"<br><b>Atoms slowed to {window_low:.0f}-{window_high:.0f} m/s</b>: {atoms_in_window:.2e} s⁻¹"
        f"<br><b>Fraction of captured atoms in window</b>: {efficiency:.2f}%"
        f"<br><b>Current intensity</b>: {intensity_val:.0f} mW/cm² | <b>Detuning</b>: {detuning_val:.2f} GHz"
    )

control_children = list(fallback_controls)
control_children.extend([temperature_slider_mb, flux_slider_mb, window_slider_mb, mb_summary_label])
controls_mb = w.VBox(control_children)
mb_output = w.interactive_output(
    estimate_atoms,
    {
        "temperature_K": temperature_slider_mb,
        "atom_flux_per_s": flux_slider_mb,
        "velocity_window": window_slider_mb,
    },
 )

display(controls_mb, mb_output)